In [23]:
import db_operations as dbop
from constants import TOKEN, STOCK_DAY, INDEX_DAY,TABLE,COLUMNS
import pandas as pd
db_type = "sqlite3"
conn = dbop.connect_db(db_type)
cursor = conn.cursor()

# Get all trading dates from stock index table.
cursor.execute("select * from {0}".format(
                    INDEX_DAY[TABLE]))
df_idx_day = pd.DataFrame(cursor.fetchall())
df_idx_day.columns = dbop.cols_from_cur(cursor)

# Get day data of all stocks from stocks table.
cursor.execute("select * from {0}".format(STOCK_DAY[TABLE]))
df_stocks_day = pd.DataFrame(cursor.fetchall())
df_stocks_day.columns = dbop.cols_from_cur(cursor)
print(df_stocks_day.shape, df_idx_day.shape)


In [19]:
print(df_stocks_day.columns)
df_stck_na = df_stocks_day[df_stocks_day.isna().any(axis=1)]
df_idx_na = df_idx_day[df_idx_day.isna().any(axis=1)]

print("indexes:",df_idx_na)
# print("stocks",df_stck_na)
print(df_stck_na.shape)
# # for code,df in df_stck_na.groupby("code"):
# #     print(df)
print(df_stocks_day.columns)

In [24]:
dates = sorted(df_idx_day["date"].unique())

In [25]:
df_stocks_day = df_stocks_day.set_index("date")
for code, df_single_stock in df_stocks_day.groupby(by="code"):
    df_single_stock = df_single_stock.sort_index()
    # start_date is the first date on which the whole row is not null in the
    #  table.
    start_date = df_single_stock[df_single_stock.notnull().all(
        axis=1)].index.min()
    print(df_single_stock.loc[start_date])
    print(df_idx_day[df_idx_day["date"]==start_date])
    start_idx = dates.index(start_date)
    for i in range(start_idx+1,len(dates)):
        if dates[i] in df_single_stock:
            if df_single_stock.loc[dates[i]].notna().all().all():
               continue
                
            is_changed = False
            row = df_single_stock.loc[dates[i]].copy()
            # Note that by using groupby code, code is always not null.
            if df_single_stock.loc[dates[i],["open","high","low","close"]].isna().all().all():
                if df_single_stock[dates[i],["vol","amt","adj_factor"]].isna().all().all():
                    # All columns except code is null.
                    df_single_stock.loc[dates[i],["open","high","low","close","adj_factor"]] = df_single_stock.loc[dates[i-1],["open","high","low","close","adj_factor"]]
                    df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
                    is_changed = True     
                elif df_single_stock[dates[i],["vol","amt"]].isna().all().all():
                    # All columns excpet code and adj_factor is null.
                    df_single_stock.loc[dates[i],["open","high","low","close"]] = df_single_stock.loc[dates[i-1],["open","high","low","close"]]
                    df_single_stock.loc[dates[i],["vol","amt"]] = [0,0]
                elif df_single_stock[dates[i],"adj_factor"].isna().all().all() \
                        and (df_single_stock[dates[i],["vol","amt"]]==[0,0]).all().all():
                    df_single_stock.loc[dates[i],["open","high","low","close","adj_factor"]] = df_single_stock.loc[dates[i-1],["open","high","low","close","adj_factor"]]
                elif df_single_stock[dates[i],["vol","amt",
                                               "adj_factor"]].notnull().all().all():
                    df_single_stock.loc[dates[i],["open","high","low","close"]] = df_single_stock.loc[dates[i-1],["open","high","low","close"]]
                else:
                    raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
                is_changed=True
            elif df_single_stock.loc[dates[i]][:-1].notna().all().all() and \
                    df_single_stock.loc[dates[i],"adj_factor"].isna().all(
                    ).all():
                df_single_stock.loc[dates[i],"adj_factor"] = df_single_stock.loc[dates[i-1],"adj_factor"]
                is_changed=True
            elif not df_single_stock.loc[dates[i]].notna().all().all():
                raise ValueError("Error row: {}".format(df_single_stock.loc[dates[i]]))
            if is_changed:
                print(row, df_single_stock[dates[i]])


code          000001.SZ
open              11.75
high              11.77
low                11.4
close             11.48
vol             63435.3
amt             73206.8
adj_factor       24.359
Name: 2002-03-22, dtype: object
Empty DataFrame
Columns: [code, date, open, high, low, close, vol]
Index: []


ValueError: '2002-03-22' is not in list